In [1]:
import pandas as pd
import os, sys, csv
import glob, errno
import numpy as np
import cv2
import copy
import math

In [2]:
def read_csv_file(filename):
    print(os.getcwd())
    csv_file=pd.read_csv(filename, header =None)
    df=pd.DataFrame(csv_file)
    df.columns=df.iloc[0]
    df=df.drop(df.index[0])
    return df

In [3]:
def normalizing_image(image_rect_dict, input_size):
    normal_height = input_size[0]
    normal_width = input_size[1]
    normalized_image_rect_dict=[]
    for document in image_rect_dict:
        
        image=document[0]
        image_info=document[1]
        shape_of_Doc = document[1][0]
        rect_info = document[1][1]
        x_factor=normal_width/(document[1][0][1])
        y_factor=normal_height/(document[1][0][0])
        document[1][0][0] = normal_height
        document[1][0][1] = normal_width
        for rect_instance_no in  range(len(document[1][1])):
            
            document[1][1][rect_instance_no][0][0] = document[1][1][rect_instance_no][0][0]*x_factor
            document[1][1][rect_instance_no][0][1] = document[1][1][rect_instance_no][0][1]*y_factor
            
            document[1][1][rect_instance_no][1][0] = document[1][1][rect_instance_no][1][0]*x_factor
            document[1][1][rect_instance_no][1][1] = document[1][1][rect_instance_no][1][1]*y_factor
            
        normalized_image_rect_dict.append(document)
    return normalized_image_rect_dict

In [4]:
def create_grid(no_of_grid, input_image_size):
    size_of_grid = (int(input_image_size[0]/no_of_grid[0]), int(input_image_size[1]/no_of_grid[1] )) #(height, width)
    list_of_grid_corner=[]
    residual_length = input_image_size[0]
    
    for row in range(no_of_grid[0]):
        bottom_right_y = 0
        
        if  residual_length >= size_of_grid[0]:
            bottom_right_y = size_of_grid[0] * (row + 1) 
        else:
            bottom_right_y = size_of_grid[0] * (row) + residual_length 
        
        residual_length = residual_length - size_of_grid[0]
        residual_width = input_image_size[1]
        row_grid=[]
        
        for col in range(no_of_grid[1]):
            bottom_right_x = 0
            
            if  residual_width >= size_of_grid[1]:
                bottom_right_x = size_of_grid[1] * (col + 1) 
            else:
                bottom_right_y = size_of_grid[1] * (col) + residual_width 
            residual_width = residual_width - size_of_grid[1]
            top_left_y = size_of_grid[0] * row
            top_left_x = size_of_grid[1] * col
            row_grid.append([(top_left_x, top_left_y), (bottom_right_x, bottom_right_y)])
            
        list_of_grid_corner.append(row_grid)
        
    return list_of_grid_corner, size_of_grid         
    

In [5]:
def isinstance_in_grid(instance, grid):

    mid_point_x = (instance[0][0] + instance[1][0])/2
    mid_point_y = (instance[0][1] + instance[1][1])/2
    
    if (mid_point_x > grid[0][0] and mid_point_x <= grid[1][0]) and (mid_point_y > grid[0][1] and mid_point_y <= grid[1][1]):
        return True
    else:
        return False
    

In [17]:
def identify_class(instance_class,no_of_class):
    #print("instance_class = ",instance_class)
    one_hot_class_vector= np.zeros((no_of_class,), dtype=int)
    class_list = ["heading", "paragraph", "table", "list", "image"]
    if instance_class != "":
        class_index = class_list.index(instance_class)
        one_hot_class_vector[class_index] = 1

    return one_hot_class_vector


In [6]:
def create_label_for_grid(instance, flag, no_of_class, label_training_instance, grid_size):
    object_presence=0
    label = np.zeros((5+no_of_class,), dtype = float)
    if flag == True and instance[4]!= "":
        object_presence = 1
        one_hot_class_vector = identify_class(instance[4], no_of_class)
        label[0] = instance[0]
        label[1] = instance[1]
        label[2] = instance[2]#width
        label[3] = instance[0]#height
        label[4] = object_presence
        label[5:] = one_hot_class_vector

    return label
        

In [7]:
def interval_overlap(interval_a, interval_b):
    x1, x2 = interval_a
    x3, x4 = interval_b

    if x3 < x1:
        if x4 < x1:
            return 0
        else:
            return min(x2,x4) - x1
    else:
        if x2 < x3:
            return 0
        else:
            return min(x2,x4) - x3  


In [8]:
def bbox_iou(box1, box2):
    x1_min  = box1[0] - box1[2]/2
    x1_max  = box1[0] + box1[2]/2
    y1_min  = box1[1] - box1[3]/2
    y1_max  = box1[1] + box1[3]/2
    
    x2_min  = box2[0] - box2[2]/2
    x2_max  = box2[0] + box2[2]/2
    y2_min  = box2[1] - box2[3]/2
    y2_max  = box2[1] + box2[3]/2
    
    intersect_w = interval_overlap([x1_min, x1_max], [x2_min, x2_max])
    intersect_h = interval_overlap([y1_min, y1_max], [y2_min, y2_max])
    
    intersect = intersect_w * intersect_h
    
    union = box1[2] * box1[3] + box2[2] * box2[3] - intersect
    
    return float(intersect) / union

In [9]:
def calculate_iou_with_true_box(instance, anchor_w, anchor_h):

    instance_cord = [0, 0, instance[0], instance[1]]
    anchor_cord = [0, 0, anchor_w, anchor_h]
    iou = bbox_iou(instance_cord, anchor_cord)
    return iou
    

In [10]:
ANCHORS = [0.57273, 0.677385, 1.87446, 2.06253]
def find_best_anchor_box(instance):
    best_anchor = -1
    max_iou     = -1
    
    for i in range(len(ANCHORS)//2):
      
        anchor_w=ANCHORS[2*i]
        anchor_h=ANCHORS[2*i + 1]
  
        iou = calculate_iou_with_true_box(instance[2:4], anchor_w, anchor_h)
    
        if max_iou < iou:
            best_anchor = i
            max_iou= iou
    return best_anchor    
    

In [11]:
def normalize_instance_relative_to_grid(no_of_grid, input_image_size, instance, grid_size):

    center_x = 0.5*no_of_grid[1]*(instance[0][0] + instance[1][0])/input_image_size[1]
    center_y = 0.5*no_of_grid[0]*(instance[0][1] + instance[1][1])/input_image_size[0]

    center_w = no_of_grid[1]*abs(instance[0][0] - instance[1][0])/input_image_size[1]
    center_h = no_of_grid[0]*abs(instance[0][1] - instance[1][1])/input_image_size[0]

    x =  (instance[0][0] + instance[1][0])/2
    y =  (instance[0][1] + instance[1][1])/2
    w = abs(instance[0][0] - instance[1][0])/grid_size[1]
    h = abs(instance[0][1] - instance[1][1])/grid_size[0]
 
    box = [center_x, center_y, w, h, instance[2]]

    return box

In [12]:
def map_each_object_t_grid(normalized_image_rect_dict, list_of_grid_corner, grid_size, no_of_class, no_of_anchor_box, input_image_size, no_of_grid ):
    label_training_set_Y = []
    label_training_set_X = []
    true_boxes = np.zeros((len(normalized_image_rect_dict),1, 1, 1, 50, 4 ))


    for image_index in range(len(normalized_image_rect_dict)):
        image_name = cv2.imread(normalized_image_rect_dict[image_index][0])
        resize_image = cv2.resize(image_name, (input_image_size[0],input_image_size[1]))#check for imahe size 
        label_training_set_X.append(resize_image)
        total_tru=0;# to check how many rect are there in any image 
       
        label_training_instance = []
       
        for row_grid_list in list_of_grid_corner:
        #    print("this is for row -------------------------------------------")
            label_training_instance_row = []
            for grid in row_grid_list:
            #    print("this is for grid in row++++++++++++++++++++++++++++")
                label_list=[None]*no_of_anchor_box
                grid_label=[]
                true_content = 0 # to check for eah anchor
                
                for instance in range(len(normalized_image_rect_dict[image_index][1][1])):
                    flag = isinstance_in_grid(normalized_image_rect_dict[image_index][1][1][instance], grid)
                    if flag == True:                    
                        #############################instance is in grid, now find best suitable anchor box for it
                        normalized_instance = normalize_instance_relative_to_grid(no_of_grid, input_image_size, normalized_image_rect_dict[image_index][1][1][instance], grid_size)
                     
                        best_anchor_index = find_best_anchor_box(normalized_instance) # give best anchor
                        if label_list[best_anchor_index] is not None:  # if best anchor is not available then assign another
                            best_anchor_index=1-best_anchor_index
                    
                        if label_list[best_anchor_index] is None and true_content< no_of_anchor_box: # anchor is available in list and all anchor is not filled 
                         
                            label = create_label_for_grid(normalized_instance, flag, no_of_class, label_training_instance, grid_size)
                           
                            label_list[best_anchor_index] = label # created label is assigned to best anchor
                         
                            true_boxes[image_index,0,0,0,total_tru] = label[:4]  # cordinates of label assgined to true box
                       
                            true_content = true_content + 1 # to check how many anchor filled 
                           
                            total_tru =total_tru+1
                
                        elif true_content < no_of_anchor_box: # when only one rect second anchor  is empty  # this time not seems to be used
                            
                           
                            zero_instance =[0,0,0,0,""] 
                            label = create_label_for_grid(zero_instance, flag, no_of_class, label_training_instance, grid_size)
                          
                            label_list[best_anchor_index] = label
                       
                            true_content = true_content + 1
                        
           
                zero_instance =[0,0,0,0,""]  # for all grid which do not have rect
                label = create_label_for_grid(zero_instance, flag, no_of_class, label_training_instance, grid_size)
              
             
                for anchor_label in range(len(label_list)):
                    if label_list[anchor_label] is None:
                        label_list[anchor_label] = label
                        true_content = true_content + 1
                       
                grid_label.append(list(label_list[0]))
                grid_label.append(list(label_list[1]))
                
            
                label_training_instance_row.append(grid_label)
                
            label_training_instance.append(label_training_instance_row)
   
       
  
        label_training_set_Y.append(label_training_instance)
    
   
    instance_y = np.asarray(label_training_set_Y)                
    instance_x = np.asarray(label_training_set_X)
    
    return instance_x, instance_y, true_boxes

        
    

In [13]:
def image_rect_printing(image_folder, image_rect_dict):

    path = os.getcwd()
    
    os.chdir(r'/home/bitnami/vikrant/deep_version/rectangled_set')

    for image in image_rect_dict:

        Image = cv2.imread(image_folder+"/"+image[0])
        cv2.imwrite(image[0],Image)
        for rect in image[1][1]:

            cv2.rectangle(Image, tuple(rect[0]), tuple(rect[1]), (0,255,0), 3)
        cv2.imwrite(image[0], Image)
    os.chdir(path)


In [122]:
def image_norm_rect_printing(image_folder, image_rect_dict, input_image_size):

    path = os.getcwd()
    
    os.chdir(r'/home/bitnami/vikrant/deep_version/rectangled_set/normalized')

    for image in image_rect_dict:

        Image = cv2.imread(image_folder+"/"+image[0])
        Image = cv2.resize(Image, (input_image_size[0], input_image_size[1])) 
        cv2.imwrite(image[0],Image)
        for rect in image[1][1]:

            cv2.rectangle(Image, (int(rect[0][0]),int(rect[0][1])), (int(rect[1][0]),int(rect[1][1])), (0,255,0), 3)
        cv2.imwrite(image[0], Image)
    os.chdir(path)


In [123]:
def IOU(data_instance, centroids):
    w, h = data_instance
    similarities = []

    for centroid in centroids:
        c_w, c_h = centroid

        if c_w >= w and c_h >= h:
            similarity = w*h/(c_w*c_h)
        elif c_w >= w and c_h <= h:
            similarity = w*c_h/(w*h + (c_w-w)*c_h)
        elif c_w <= w and c_h >= h:
            similarity = c_w*h/(w*h + c_w*(c_h-h))
        else: #means both w,h are bigger than c_w and c_h respectively
            similarity = (c_w*c_h)/(w*h)
        similarities.append(similarity) # will become (k,) shape

    return np.array(similarities)

In [128]:
import random
def run_kmean(kmean_data_instance,no_of_anchor_box ):
    no_instances = kmean_data_instance.shape[0]
    iterations = 0
    prev_assignments = np.ones(no_instances)*(-1)
    iteration = 0
    old_distances = np.zeros((no_instances, no_of_anchor_box))

    indices = [random.randrange(kmean_data_instance.shape[0]) for i in range(no_of_anchor_box)]
    centroids = kmean_data_instance[indices]
    anchor_dimesion = kmean_data_instance.shape[1]
    ########################### do changes from here
    while True:
        distances = []
        iteration += 1
        for i in range(no_instances):
            d = 1 - IOU(kmean_data_instance[i], centroids)
            distances.append(d)
        distances = np.asarray(distances) # distances.shape = ( no_instances, no_of_anchor_box)

       # print("iteration {}: dists = {}".format(iteration, np.sum(np.abs(old_distances-distances))))

        #assign samples to centroids
        assignments = np.argmin(distances,axis=1)

        if (assignments == prev_assignments).all() :
            return centroids

        #calculate new centroids
        centroid_sums=np.zeros((no_of_anchor_box, anchor_dimesion), np.float)
        for i in range(no_instances):
            centroid_sums[assignments[i]]+=kmean_data_instance[i]
        for j in range(no_of_anchor_box):
            centroids[j] = centroid_sums[j]/(np.sum(assignments==j) + 1e-6)

        prev_assignments = assignments.copy()
        old_distances = distances.copy()    
    return centroids
    
    

In [129]:
def create_anchor_box(normalized_image_rect_dict, no_of_anchor_box, no_of_grid, input_image_size):
 #   print("no_of_anchor_box = ",no_of_anchor_box)
 #   print("len(normalized_image_rect_dict) =", len(normalized_image_rect_dict))
 #   print("no_of_grid = ",no_of_grid)
 #   print("input_image_size = ", input_image_size)
 #   no_of_image = len(normalized_image_rect_dict)
    cell_w = input_image_size[1]/ no_of_grid[1]
    cell_h = input_image_size[0]/ no_of_grid[0]
    count = 0
    total_rect_wh_in_dataset = []
    for image in  normalized_image_rect_dict:
        image_name = image[0]
       # print("image_name",image_name)
        total_rect_in_image = len(image[1][1])
        count = count +  total_rect_in_image
      #  print("total_rect_in_image ",image_name, total_rect_in_image)
        #print("\n\n")
        for rect in image[1][1]:
            top_left  = rect[0]
            bottom_right = rect[1]
            rect_class = rect[2]
            #print("top, bottom, class = ",top_left,bottom_right, rect_class )
            relative_w = float(abs(bottom_right[0] - top_left[0])/cell_w)
            relative_h = float(abs(bottom_right[1] - top_left[1])/cell_h)
            #print("w ", abs(bottom_right[0] - top_left[0]))
           # print("h ", abs(bottom_right[1] - top_left[1]))
            
            total_rect_wh_in_dataset.append(tuple((relative_w, relative_h)))
            
  #  print("count = ",count)
   # print("len(total_rect_wh_in_dataset) = ",len(total_rect_wh_in_dataset))
    kmean_data_instance = np.asarray(total_rect_wh_in_dataset)
   # print("kmean_datainstance.shape ",kmean_data_instance.shape )
    centroids = run_kmean(kmean_data_instance,no_of_anchor_box )
    print("centroids = ",centroids)
    return centroids
    

In [130]:
def create_train_set(path, file, image_folder, input_image_size, no_of_grid, no_of_class, no_of_anchor_box ):
    os.getcwd()

    os.chdir(path)
    os.getcwd()
    df = read_csv_file(file)
    image_list=df[['Image_name','Image_shape']].drop_duplicates()

    os.chdir(image_folder)
    
    
    ###############################
    image_rect_dict=[]
    i=0;
    total_instances=0
    no_of_instances = len(image_list['Image_name'])
  
    for img in image_list['Image_name']:
    
      
        orig_image= cv2.imread(img)
 
        image_shape=image_list.iloc[i]['Image_shape']
        image_shape=[int(item) for item in image_list.iloc[i]['Image_shape'][1:][:-1].split(",")]
       
        
        i=i+1;
        rect_midpoint=[]
     
        
        rect_midpoint_index=df.index[df['Image_name'] == img].tolist()
    
        Image_info=[]
        for rect in rect_midpoint_index:
            rect_info=[ df.iloc[rect-1]['Top_left'] , df.iloc[rect-1]['Bottom_Right'], df.iloc[rect-1]['class']]
            Top_left=[int(item) for item in df.iloc[rect-1]['Top_left'][1:][:-1].split(",")]
            Bottom_Right=[int(item) for item in df.iloc[rect-1]['Bottom_Right'][1:][:-1].split(",")]
            
            Image_info.append([Top_left, Bottom_Right, df.iloc[rect-1]['class']])
        
    
        total_instances+=len(Image_info)
        image_rect_dict.append([img,[image_shape, Image_info]])

    #########################################
    #image rect printing before normalization
   # print("creating rectangled dataset")
   # image_rect_printing(image_folder, image_rect_dict)
        
    Image_rect_dict = copy.deepcopy(image_rect_dict)
   # print("creating rectangled normalized dataset")
    normalized_image_rect_dict = normalizing_image(image_rect_dict, input_image_size)
    #########################################
    #image rect printing before normalization
    #image_norm_rect_printing(image_folder, normalized_image_rect_dict, input_image_size)
    #create_anchr_box()#$#$#$#$#$#$#$@#$@#$#$#$#$@#$@#$#$#%$#$
    
    
    
    #uncooment and run to generate anchore box
    anchor_boxes = create_anchor_box(normalized_image_rect_dict, no_of_anchor_box, no_of_grid, input_image_size)
    list_of_grid_corner, grid_size = create_grid(no_of_grid , input_image_size )
    
    instance_x, instance_y,true_boxes = map_each_object_t_grid(normalized_image_rect_dict, list_of_grid_corner, grid_size, no_of_class , no_of_anchor_box , input_image_size ,no_of_grid )

    return instance_x, instance_y, true_boxes, anchor_boxes 
    

In [131]:
instance_x, instance_y, true_boxes, anchor_boxes = create_train_set(path = r'/home/bitnami/vikrant/deep_version/extraction_code', file = 'train_set.csv', image_folder = r'/home/bitnami/vikrant/deep_version/went_into_csv', input_image_size = (1024, 1024), no_of_grid = (8, 8), no_of_class = 5, no_of_anchor_box = 2 )
#print("instance_x.shape = ",instance_x.shape)
#print("instance_y.shape = ",instance_y.shape)
#print("true_boxes.shape = ",true_boxes.shape)

/home/bitnami/vikrant/deep_version/extraction_code
centroids =  [[1.90603223 0.14906887]
 [5.53059165 0.74886134]]
